# ezSegmenter

#### This segmentation tool enables creation of masks for objects not easily picked up by primary cell segmentation methods on multiplexed imaging data.
#### In addition this tool can be used to create composites of channels as well as merge object masks with cell masks.

## 0. Imports and Path Setting

In [ ]:
%load_ext autoreload

In [5]:
%autoreload 2

In [6]:
import pathlib
from ark.segmentation import ez_object_segmentation, merge_masks, composite_builder
from alpineer import load_utils
from tqdm.notebook import tqdm
import xarray as xr

In [16]:
# List the path where your multiplexed imaging data lives. The folder should contain subfolders for each FoV you wish to work upon.
base_dir = pathlib.Path("../../data/experiments/ezSeg Data/")
point77_TIFs = base_dir / "Point77" / "TIFs"
p77_tiffs = list((point77_TIFs).glob("*.tiff"))

In [19]:
# Editing path / tiff names
for p in p77_tiffs:
    # remove whitespace from filename
    p_replace = p.name.replace(" ", "")
    p.rename(p.with_name(p_replace).with_suffix(".tiff"))

In [ ]:
# Load tiff files as xarray
p77_xr: xr.DataArray = load_utils.load_imgs_from_dir(data_dir=point77_TIFs).drop_vars("compartments").squeeze()
p77_xr

1. Create object masks
2. Composite builder
3. Merge masks

## 1. Create Object Masks

In [21]:
object_masks: xr.DataArray = xr.zeros_like(other=p77_xr)
object_masks.name="Object Masks"
for fov in tqdm(p77_xr.fovs.values):
    fov_2d = p77_xr.loc[fov].values
    object_masks.loc[fov].values = ez_object_segmentation.create_object_masks(input_image=fov_2d)

  0%|          | 0/47 [00:00<?, ?it/s]

In [22]:
object_masks.fovs.values

array(['8OHGuano', 'Amyloidbeta140', 'Amyloidbeta142', 'ApoE4', 'Au197',
       'Background', 'C12', 'CD105', 'CD31', 'CD33Lyo', 'CD45', 'CD47',
       'CD56Lyo', 'Ca40', 'Calbindin', 'Calretinin', 'EEA1', 'GFAP',
       'HistoneH3Lyo', 'Iba1', 'MAG', 'MAP2', 'MBP', 'MCT1', 'MFN2',
       'Na23', 'PHF1Tau', 'PSD95', 'PanAmyloidbeta1724', 'PanApoE2E3E4',
       'PanGAD6567', 'Parvalbumin', 'PolyubiK48', 'PolyubiK63',
       'Presenilin1NTF', 'Reelin', 'SERT', 'Si28', 'Synaptophysin', 'TH',
       'Ta181', 'TotalTau', 'VGAT', 'VGLUT1', 'VGLUT2', 'empty139',
       'pTDP43'], dtype='<U18')

## 2. Composite Builder

In [45]:
composite_directory = "../../data/experiments/ezSeg Data/Composites/Point77/"

composite_name = "composite_a.tiff"

In [46]:
composite_array = composite_builder.composite_builder(
    data=object_masks,
    images_to_add=["8OHGuano", "C12", "Reelin"],
    images_to_subtract=["MBP", "Iba1", "Si28"],
    composite_name = composite_name,
    composite_directory=composite_directory,
    image_type="intensity",
    composite_method="full",
)
composite_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint64)

## 3. Mask Merger

**Note:** Requires the Deepcell outputs from `1_Segment_Image_Data.ipynb`.

In [25]:
curr_object_mask = pathlib.Path("../../data/experiments/ezSeg Data/segmentation/deepcell_output/TIFs_whole_cell.tiff")

In [62]:
merge_masks.merge_masks_seq(
    object_list=["composite_b", "composite_a"],
    object_mask_dir=composite_directory,
    cell_mask_path=curr_object_mask,
    overlap=10,
    save_path=composite_directory,
)